In [0]:
from pyspark.sql.functions import (col, trim, initcap, when, current_timestamp)

In [0]:
# lendo DataFrame da tabela produtos na camada silver

produtos_df = spark.table("ecommerce.bronze.produtos")

In [0]:
produtos_df = (
    produtos_df
    # Remover espaços extras
    .withColumn("nome", trim(col("nome")))
    .withColumn("marca", initcap(trim(col("marca"))))
    
    # Converter preço
    .withColumn("preco", col("preco").cast("decimal(10,2)"))
    
    # Criar faixa de preço
    .withColumn(
        "faixa_preco",
        when(col("preco") < 100, "Econômico")
        .when((col("preco") >= 100) & (col("preco") <= 500), "Médio")
        .when(col("preco") > 500, "Premium")
        .otherwise("Indefinido")
    )
    
    # Filtrar produtos ativos e válidos
    .filter(
        (col("ativo") == True) &
        (col("preco").isNotNull()) &
        (col("preco") > 0)
    )
)

In [0]:
produtos_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("ecommerce.silver.produtos")
